In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
source = '..\\Applications\\intruder_2.mp4'
video_cap = cv2.VideoCapture(source)
if not video_cap.isOpened():
    print('Unable to load the Video')  

In [3]:
frame_w = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video_cap.get(cv2.CAP_PROP_FPS))

In [4]:
size = (frame_w,frame_h)

In [5]:
frame_area = frame_w * frame_h

In [6]:
video_out_alert_file_2 = 'video_out_alert_2.mp4'
video_out_alert_2 = cv2.VideoWriter(video_out_alert_file_2,cv2.VideoWriter_fourcc(*'mp4v'),fps,size)

In [7]:
bg_sub = cv2.createBackgroundSubtractorKNN(200)

In [8]:
ksize = (5,5)
max_contours = 3
frame_count = 0
min_contour_area_thresh = 0.01
yellow = (0,255,255)
red = (0,0,255)

In [28]:
def drawBannerText(frame,text,banner_height_percent = 0.08,font_scale = 0.8, text_color=(0, 255, 0), font_thickness=2):
    banner_height = int(banner_height_percent * frame.shape[0])
    cv2.rectangle(frame,(0,0),(frame.shape[1],banner_height),(0,0,0),thickness=-1)
    left_offset = 20
    location = (left_offset,int(10 + int(banner_height_percent * frame.shape[0])/2))
    cv2.putText(frame,text,location,cv2.FONT_HERSHEY_SIMPLEX,font_scale,text_color,font_thickness,cv2.LINE_AA)

In [29]:
while True:
    ret,frame = video_cap.read()
    frame_count+=1
    if frame is None:
        break
    fg_mask = bg_sub.apply(frame)

    fg_mask_erode = cv2.erode(fg_mask,np.ones(ksize,dtype='uint8'))

    contour_erode, hierarchy = cv2.findContours(fg_mask_erode,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)

    if len(contour_erode) > 0:
        contour_sorted = sorted(contour_erode,key= cv2.contourArea,reverse=True)
    contour_area_max = cv2.contourArea(contour_sorted[0])

    contour_frac = contour_area_max/frame_area

    if contour_frac > min_contour_area_thresh:
        for idx in range(min(max_contours,len(contour_erode))):
            xc,yc,wc,hc = cv2.boundingRect(contour_sorted[idx])
            if idx == 0:
                x1 = xc
                y1 = yc
                x2 = wc+xc
                y2 = hc+yc
            else:
                x1 = min(x1,xc)
                y1 = min(y1,yc)
                x2 = max(x2,xc+wc)
                y2 = max(y2,yc+hc)
        cv2.rectangle(frame,(x1,y1),(x2,y2),yellow,thickness= 2)
        drawBannerText(frame,'Intrusion Video',text_color=red)
        video_out_alert_2.write(frame)

video_cap.release()
video_out_alert_2.release()

        
                
    

    
    

In [20]:
from moviepy import VideoFileClip

In [30]:
clip = VideoFileClip(video_out_alert_file_2)
clip.display_in_notebook(width=1000)       

{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2mp41', 'encoder': 'Lavf58.76.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': 3651, 'fps': 30.0, 'codec_name': 'mpeg4', 'profile': '(Simple Profile)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 6.53, 'bitrate': 3653, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'mpeg4', 'video_profile': '(Simple Profile)', 'video_size': [1280, 720], 'video_bitrate': 3651, 'video_fps': 30.0, 'video_duration': 6.53, 'video_n_frames': 195}
C:\Users\dell\miniconda3\envs\opencv-env\Lib\site-packages\imageio_ffmpeg\binaries\ffmpeg-win-x86_64-v7.1.exe -i video_out_alert_2.mp4 -loglevel error -f image2pipe -vf scale=1280:720 -sws_f

MoviePy - Done !
MoviePy - video ready __temp__.mp4
